In [1]:
!pip install transformers
!pip install datasets
!pip install tensorflow
!pip install pdfminer.six
!pip install nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 31.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_dataset
import tensorflow as tf
from pdfminer.high_level import extract_text # type: ignore
import nltk # type: ignore
import pandas as pd # type: ignore
import re

# Extract Text and Preprocessing


In [4]:
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

In [5]:
text=extract_text_from_pdf('/content/drive/MyDrive/Dl-project/Sample-Resume.pdf')
#text="A leading real estate firm in New Jersey seeks an administrative Marketing Coordinator with graphic design experience. You'll work closely with the sales and executive teams, contributing to a fast-growing firm known for exceptional marketing. Responsibilities include agent assistance, graphic design and branding, and event planning. The ideal candidate is organized, creative, positive, and skilled in Adobe Creative Cloud and Microsoft Office Suite. This is a full-time position with a commitment to diversity and equal opportunity."
print(text)


JACQUELINE THOMPSON

123 Anywhere St., Any City • 123-456-7890 • hello@reallygreatsite.com
www.reallygreatsite.com

SUMMARY

Results-oriented Engineering Executive with a proven track record of optimizing project outcomes.
Skilled in strategic project management and team leadership. Seeking a challenging executive role
to leverage technical expertise and drive engineering excellence.

WORK EXPERIENCE

Engineering Executive , Borcelle Technologies

Jan 2023 - Present

Implemented cost-effective solutions, resulting in a 20% reduction in project expenses.
Streamlined project workflows, enhancing overall efficiency by 25%.
Led a team in successfully delivering a complex engineering project on time and within
allocated budget.

Project Engineer, Salford & Co

Mar 2021 - Dec 2022

Managed project timelines, reducing delivery times by 30%.
Spearheaded the adoption of cutting-edge engineering software, improving project
accuracy by 15%.
Collaborated with cross-functional teams, enhancing proj

In [6]:
import re
import string


resume_text_cleaned=text

#Remove URLs (links) that start with http, https, or www
resume_text_cleaned = re.sub(r'(https?://\S+|www\.\S+)', '', resume_text_cleaned)

resume_text_cleaned = re.sub(r'\S+@\S+', '', resume_text_cleaned)

resume_text = re.sub(r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{4}\b', '', text)


resume_text_cleaned = re.sub(r'\t+', ' ', resume_text_cleaned).strip()


punctuation_to_remove = string.punctuation.replace('@', '')


print(resume_text_cleaned)

JACQUELINE THOMPSON

123 Anywhere St., Any City • 123-456-7890 • 


SUMMARY

Results-oriented Engineering Executive with a proven track record of optimizing project outcomes.
Skilled in strategic project management and team leadership. Seeking a challenging executive role
to leverage technical expertise and drive engineering excellence.

WORK EXPERIENCE

Engineering Executive , Borcelle Technologies

Jan 2023 - Present

Implemented cost-effective solutions, resulting in a 20% reduction in project expenses.
Streamlined project workflows, enhancing overall efficiency by 25%.
Led a team in successfully delivering a complex engineering project on time and within
allocated budget.

Project Engineer, Salford & Co

Mar 2021 - Dec 2022

Managed project timelines, reducing delivery times by 30%.
Spearheaded the adoption of cutting-edge engineering software, improving project
accuracy by 15%.
Collaborated with cross-functional teams, enhancing project success rates by 10%.

Graduate Engineer, Ar

# Importing the model ( BART )




In [ ]:
checkpoint = 'facebook/bart-large-xsum' # Model
tokenizer = BartTokenizer.from_pretrained(checkpoint) # Loading Tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

In [ ]:
model = BartForConditionalGeneration.from_pretrained(checkpoint) # Loading Model

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

In [ ]:
print(model)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

# Training Summarizer Model


In [ ]:
dataset = load_dataset("AjayMukundS/Legal_Text_Summarization-llama2")

train_test_split = dataset['train'].train_test_split(test_size=0.2)  # 80% train, 20% test

# Further split the test set into validation and test sets
test_valid_split = train_test_split['test'].train_test_split(test_size=0.5)  # 50% validation, 50% test

README.md:   0%|          | 0.00/494 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/107M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/11.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7773 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset, DatasetDict


split_dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': test_valid_split['train'],
    'test': test_valid_split['test'],
})

In [ ]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["judgement"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# Assuming you have already created `split_dataset` containing train, validation, and test sets

# Preprocess the training dataset
tokenized_train = split_dataset['train'].map(
    preprocess_function,
    batched=True,
    remove_columns=['judgement','dataset_name', 'text', 'summary']
)

# Preprocess the validation dataset
tokenized_val = split_dataset['validation'].map(
    preprocess_function,
    batched=True,
    remove_columns=['judgement','dataset_name', 'text', 'summary']
)

# Preprocess the test dataset
tokenized_test = split_dataset['test'].map(
    preprocess_function,
    batched=True,
    remove_columns=['judgement','dataset_name', 'text', 'summary']
)

# Print the structure of the tokenized datasets for verification
print("Tokenized Training Dataset:", tokenized_train)
print("Tokenized Validation Dataset:", tokenized_val)
print("Tokenized Test Dataset:", tokenized_test)


Map:   0%|          | 0/6218 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/777 [00:00<?, ? examples/s]

Map:   0%|          | 0/778 [00:00<?, ? examples/s]

Tokenized Training Dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 6218
})
Tokenized Validation Dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 777
})
Tokenized Test Dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 778
})


In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset


training_args = Seq2SeqTrainingArguments(
    output_dir = 'fine-tuned',
    evaluation_strategy = "epoch",
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    metric_for_best_model = 'eval_loss',
    seed = 42,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,1.853000,1.623624
2,1.290500,1.541639
3,1.159200,1.543877


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a G

TrainOutput(global_step=3108, training_loss=1.423395031667584, metrics={'train_runtime': 3588.5944, 'train_samples_per_second': 6.931, 'train_steps_per_second': 0.866, 'total_flos': 5.38698861993001e+16, 'train_loss': 1.423395031667584, 'epoch': 3.997427652733119})

In [ ]:
# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/Dl-project/fine_tuned_resume_summarizer")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Dl-project/fine_tuned_resume_summarizer")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


('/content/drive/MyDrive/Dl-project/fine_tuned_resume_summarizer/tokenizer_config.json',
 '/content/drive/MyDrive/Dl-project/fine_tuned_resume_summarizer/special_tokens_map.json',
 '/content/drive/MyDrive/Dl-project/fine_tuned_resume_summarizer/vocab.json',
 '/content/drive/MyDrive/Dl-project/fine_tuned_resume_summarizer/merges.txt',
 '/content/drive/MyDrive/Dl-project/fine_tuned_resume_summarizer/added_tokens.json')

# Summarization

In [7]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load your fine-tuned BART model and tokenizer
model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/Dl-project/fine_tuned_resume_summarizer')
tokenizer = BartTokenizer.from_pretrained('/content/drive/MyDrive/Dl-project/fine_tuned_resume_summarizer')

# Define the summarizer function
def summarizer(text, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=length_penalty,
                                 num_beams=num_beams, early_stopping=early_stopping)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary



In [8]:
summary = summarizer(resume_text_cleaned , max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)

# Print the summary
print("Summary of the Resume:")
print(summary)


Summary of the Resume:
JACQUELINE THOMPSON is a results-oriented Engineering Executive with a proven track record of optimizing project outcomes.
Skilled in strategic project management and team leadership, she is ideally suited to a challenging executive role to leverage technical expertise and drive engineering excellence.
She has extensive experience in project management, advanced manufacturing, and project analysis.
Her preferred subject area is structural engineering.
JACQUIELINE's resume includes: (i) a master's degree in Mechanical Engineering from the University of Engineering and Technology, London; (ii) a postgraduate degree in Civil Engineering from City College of Engineering


# Fine tuning Text-Similarity model(SBERT)

In [ ]:
!pip install transformers sentence-transformers scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 6.9 MB/s eta 0:00:00


In [ ]:
import logging
import os
import sys
import traceback
from datetime import datetime

from datasets import load_dataset
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.similarity_functions import SimilarityFunction
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# Set the log level to INFO to get more information
logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO)

# Model details
model_name = 'all-MiniLM-L6-v2'
train_batch_size = 16
num_epochs = 4

os.environ["WANDB_DISABLED"] = "true"

# Specify output directory
output_dir = "output/training_stsbenchmark_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# 1. Define SentenceTransformer model
model = SentenceTransformer(model_name)

# 2. Load the STSB dataset
train_dataset = load_dataset("sentence-transformers/stsb", split="train")
eval_dataset = load_dataset("sentence-transformers/stsb", split="validation")

# Convert to list of tuples for the evaluator
eval_sentences1 = list(eval_dataset["sentence1"])
eval_sentences2 = list(eval_dataset["sentence2"])
eval_scores = list(eval_dataset["score"])

# 3. Define the training loss
train_loss = losses.CosineSimilarityLoss(model=model)

# 4. Define an evaluator for evaluation during training
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_sentences1,
    sentences2=eval_sentences2,
    scores=eval_scores,
    main_similarity=SimilarityFunction.COSINE,
)

# 5. Define the training arguments
args = SentenceTransformerTrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=train_batch_size,
    warmup_ratio=0.1,
    fp16=True,  # Set False if GPU does not support FP16
    bf16=False,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
)

# 6. Create the trainer and start training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler 

Step,Training Loss,Validation Loss,Pearson Cosine,Spearman Cosine,Pearson Manhattan,Spearman Manhattan,Pearson Euclidean,Spearman Euclidean,Pearson Dot,Spearman Dot,Pearson Max,Spearman Max
100,0.025800,0.023121,0.888676,0.885877,0.883710,0.885433,0.884082,0.885877,0.888676,0.885877,0.888676,0.885877
200,0.022900,0.021408,0.893324,0.891578,0.888017,0.890951,0.888508,0.891578,0.893324,0.891578,0.893324,0.891578
300,0.022200,0.020284,0.894232,0.892423,0.887419,0.892530,0.887512,0.892423,0.894232,0.892423,0.894232,0.892530
400,0.017800,0.021273,0.893669,0.892638,0.888552,0.892352,0.888743,0.892638,0.893669,0.892638,0.893669,0.892638
500,0.013500,0.021060,0.892699,0.892428,0.887391,0.892466,0.887703,0.892428,0.892699,0.892428,0.892699,0.892466
600,0.012300,0.021517,0.893295,0.892081,0.889055,0.891706,0.889349,0.892081,0.893295,0.892081,0.893295,0.892081
700,0.012800,0.020771,0.892627,0.891031,0.886344,0.890632,0.886573,0.891031,0.892627,0.891031,0.892627,0.891031
800,0.009000,0.020715,0.895123,0.894123,0.889110,0.893640,0.889409,0.894123,0.895123,0.894122,0.895123,0.894123
900,0.008000,0.020820,0.895204,0.894318,0.889608,0.893877,0.889845,0.894318,0.895204,0.894318,0.895204,0.894318
1000,0.007500,0.020860,0.894740,0.894321,0.889437,0.894093,0.889686,0.894321,0.894740,0.894321,0.894740,0.894321


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=1440, training_loss=0.012545340901447667, metrics={'train_runtime': 154.7, 'train_samples_per_second': 148.649, 'train_steps_per_second': 9.308, 'total_flos': 0.0, 'train_loss': 0.012545340901447667, 'epoch': 4.0})

In [ ]:
model.save('/content/drive/MyDrive/Dl-project/fine_tuned_similarity_model')

In [ ]:
import os

# Path to your fine-tuned model in Google Drive (adjust the path accordingly)
model_path = '/content/drive/MyDrive/Dl-project/fine_tuned_similarity_model'  # Update this path

# Get the total size of the model directory (in bytes)
model_size = sum(os.path.getsize(os.path.join(root, file)) for root, dirs, files in os.walk(model_path) for file in files)

# Convert bytes to MB
model_size_mb = model_size / (1024 * 1024)
print(f"Model size: {model_size_mb:.2f} MB")


Model size: 87.58 MB


# Similarity Scores

In [9]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the SBERT model
model = SentenceTransformer('/content/drive/MyDrive/Dl-project/fine_tuned_similarity_model')
print(model)

# Sample summarized resume and job description
summarized_resume =summary
job_descriptions = [
    "Looking for a data scientist proficient in machine learning, deep learning, and data analysis.",
    "Hiring a software engineer with experience in cloud technologies like AWS.",
    "Seeking a project manager with strong skills in Agile methodologies and team leadership.",
    "Seeking a Data Scientist/ML Engineer with expertise in AI, Python, Flask, MySQL, machine learning, and NLP for innovative projects.",
    "Lead engineering projects, optimize outcomes, and mentor teams. Requires strong project management skills, expertise in structural engineering, and a results-oriented mindset."
]

resume_embedding = model.encode(summarized_resume)
job_embeddings = model.encode(job_descriptions)

resume_embedding = np.reshape(resume_embedding, (1, -1))
job_embedding = np.reshape(job_embeddings, (1, -1))

# Compute cosine similarities for each job description
similarity_scores = cosine_similarity(resume_embedding, job_embeddings)

# Print the similarity scores
for i, score in enumerate(similarity_scores[0]):
    print(f"Job {i+1} Similarity Score: {score*100}")


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
Job 1 Similarity Score: 16.904784739017487
Job 2 Similarity Score: 31.811589002609253
Job 3 Similarity Score: 51.4046311378479
Job 4 Similarity Score: 36.35576069355011
Job 5 Similarity Score: 65.58379530906677


In [10]:
best_match_idx = np.argmax(similarity_scores)
best_match_score = similarity_scores[0][best_match_idx]
best_match_description = job_descriptions[best_match_idx]

print(f"Best Matching Job Description: {best_match_description}")
print(f"Best Matching Similarity Score: {best_match_score}")


Best Matching Job Description: Lead engineering projects, optimize outcomes, and mentor teams. Requires strong project management skills, expertise in structural engineering, and a results-oriented mindset.
Best Matching Similarity Score: 0.6558379530906677
